In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

C:\Users\HRUSHIKESH\AppData\Local\Temp\ipykernel_166572\368771584.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# chunksize = 1000

# # Open the JSON file in chunks
# chunks = pd.read_json("Toys_and_Games.json", lines=True, chunksize=chunksize)

# # Initialize an empty list to store the chunks
# chunk_list = []

# # Process each chunk
# for chunk in chunks:
#     # Append the current chunk to the chunk_list
#     chunk_list.append(chunk)

# # Concatenate the chunks into a single DataFrame
# combined_df = pd.concat(chunk_list, ignore_index=True)

# # Now you have a single DataFrame containing all the data from the JSON file
# print(combined_df.head())


In [2]:
df = pd.read_csv(r'E:\Git Demo\CDAC_Group_Project\Project_dataset\toys.csv')

In [12]:
x = df['reviewText']
y = df['class']

In [7]:
# # Tokenization
# max_words = 10000
# max_len = 1000
# tokenizer = Tokenizer(num_words=max_words)
# tokenizer.fit_on_texts(X)
# X = tokenizer.texts_to_sequences(X)
# X = pad_sequences(X, maxlen=max_len)

In [16]:
text_dataset = tf.data.Dataset.from_tensor_slices(x)

In [14]:
# # Train-test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# train_data , test_data = train_test_split(x,y,test_size=0.2,random_state=0)

In [17]:
max_features = 10000
max_len = 1000
vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

vectorize_layer.adapt(text_dataset.batch(64))

In [18]:
vocab = np.array(vectorize_layer.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'it', 'to', 'a', 'i', 'is', 'this',
       'for', 'of', 'my', 'with', 'in', 'that', 'was', 'but', 'not', 'on'],
      dtype='<U15')

In [9]:
# # Model Building
# model = Sequential()
# model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
# model.add(LSTM(64))
# model.add(Dense(1, activation='sigmoid'))

In [19]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [20]:
# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
# Model Training
batch_size = 64
epochs = 5
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/5
3750/3750 [==============================] - 250s 64ms/step - loss: 0.5276 - accuracy: 0.8045 - val_loss: 7.6691 - val_accuracy: 0.4990
Epoch 2/5
3750/3750 [==============================] - 277s 74ms/step - loss: 0.6224 - accuracy: 0.8237 - val_loss: 0.3713 - val_accuracy: 0.8479
Epoch 3/5
3750/3750 [==============================] - 281s 75ms/step - loss: 0.5423 - accuracy: 0.8327 - val_loss: 0.3944 - val_accuracy: 0.8614
Epoch 4/5
3750/3750 [==============================] - 285s 76ms/step - loss: 0.3675 - accuracy: 0.8652 - val_loss: 0.3851 - val_accuracy: 0.8664
Epoch 5/5
3750/3750 [==============================] - 263s 70ms/step - loss: 0.3681 - accuracy: 0.8621 - val_loss: 0.3851 - val_accuracy: 0.8738


In [23]:
print(model.history.history)
loss_df = pd.DataFrame(model.history.history)

{'loss': [0.5276248455047607, 0.6224164962768555, 0.5423261523246765, 0.367537260055542, 0.3680608868598938], 'accuracy': [0.8045333623886108, 0.8236791491508484, 0.8327333331108093, 0.8652125000953674, 0.8621041774749756], 'val_loss': [7.669101715087891, 0.3712904155254364, 0.39440178871154785, 0.38506630063056946, 0.38514769077301025], 'val_accuracy': [0.49896666407585144, 0.8479333519935608, 0.8613666892051697, 0.8664000034332275, 0.8738499879837036]}


In [24]:
# Model Evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

1875/1875 [==============================] - 39s 21ms/step - loss: 0.3851 - accuracy: 0.8738
Test Accuracy: 0.8738499879837036


In [25]:
loss_df

,loss,accuracy,val_loss,val_accuracy
0,0.527625,0.804533,7.669102,0.498967
1,0.622416,0.823679,0.371290,0.847933
2,0.542326,0.832733,0.394402,0.861367
3,0.367537,0.865213,0.385066,0.866400
4,0.368061,0.862104,0.385148,0.873850


In [26]:
model.save_weights('LSTM_02_model_weights.h1')